In [ ]:
#Insert Songs
import os
import mysql.connector
from mutagen import File

# MySQL database configuration
db_config = {
    'user': 'root',
    'password': '',
    'host': 'localhost',
    'port': 3308,
    'database': 'spotify-clone'
}
def title_exists(cursor, title):
    query = "SELECT COUNT(*) FROM songs WHERE name = %s"
    cursor.execute(query, (title,))
    count = cursor.fetchone()[0]
    return count > 0  # Return True if exists, False otherwise

# Function to insert song details into MySQL
def insert_song(cursor, sid, title, artists, location, album, duration, icon_path):
    query = "INSERT INTO songs (id, name, artist, location, album, duration, image) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(query, (sid, title, artists, location, album, duration, icon_path))

# Paths to the folders
playlist_folder = "C:\\Users\\deeks\\Desktop\\Spotify_clone\\imgs\\classical_playlist"
icons_folder = "C:\\Users\\deeks\\Desktop\\Spotify_clone\\imgs\\songs_icons"

# Connect to MySQL database
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()
query = "select max(id) from songs"
cursor.execute(query)
last_id=cursor.fetchone()[0]
if not last_id:
    last_id=0
last_id+=1
# Iterate through each file in the classical playlist folder
for filename in os.listdir(playlist_folder):
    if filename.endswith(".mp3") or filename.endswith(".wav"):  # Add more formats as needed
        audio_file_path = os.path.join(playlist_folder, filename)
        audio = File(audio_file_path)

        if audio is not None:
            title = audio.get('TIT2', [None])[0]
            artists = audio.get('TPE1', [None])[0]
            if artists is not None:
                artists=artists.replace('/',',')
            else:
                artists='Unknown'
            album = audio.get('TALB', [None])[0]
            if album is None:
                album='Unknown'
            duration = audio.info.length  # Duration in seconds
            minutes=int(duration//60)
            secs=int(duration%60)
            duration=str(minutes)+':'+str(secs)
            icon_path=''
            for imagename in os.listdir(icons_folder):
                li=imagename.split('.')
                imagename=li[0]
                ext=li[1]
                if imagename.lower() in filename.lower():
                    print(imagename.lower(), filename.lower())
                    icon_path = os.path.join(icons_folder, imagename)+'.'+ext
                    break
            if not title:
                title=filename.lower()[:-4].title()
            # Insert the song details into the database
            if title and not title_exists(cursor, title):
                insert_song(cursor, last_id, title, artists, os.path.join(playlist_folder, filename), album, duration, icon_path)
                last_id+=1
            else:
                print('Skipping')

# Commit the transaction and close the connection
connection.commit()
cursor.close()
connection.close()

print("Data inserted successfully!")


a broken heart a broken heart.mp3
ami je tomar ami je tomar (from _bhool bhulaiyaa 2_).mp3
ami je tomar ami je tomar (kartik x vidya).mp3
bairi piya bairi piya - from _devdas_.mp3
bhari bhari bhari bhari.mp3
deewani mastani deewani mastani.mp3
dola re dola re dola - from _devdas_.mp3
ghoomar ghoomar.mp3
mere dholna mere dholna (the sisters).mp3
mere dholna mere dholna revisited.mp3
nagada sang nagada sang dhol.mp3
nature sounds nature sounds.mp3
pinga pinga.mp3
silsila ye silsila ye chahat ka - from _devdas_.mp3
tere naina tere naina maar hi daalenge (from _jai ho_).mp3
Data inserted successfully!


In [31]:
#Insert Songs
import os
import mysql.connector
from mutagen import File

# MySQL database configuration
db_config = {
    'user': 'root',
    'password': '',
    'host': 'localhost',
    'port': 3308,
    'database': 'spotify-clone'
}
def title_exists(cursor, title):
    query = "SELECT COUNT(*) FROM artists WHERE name = %s"
    cursor.execute(query, (title,))
    count = cursor.fetchone()[0]
    return count > 0  # Return True if exists, False otherwise

# Function to insert song details into MySQL
def insert_artist(cursor, sid, title, icon_path):
    query = "INSERT INTO artists (id, name, image) VALUES (%s, %s, %s)"
    cursor.execute(query, (sid, title, icon_path))

# Paths to the folders
icons_folder = "C:\\Users\\deeks\\Desktop\\Spotify_clone\\imgs\\artist_icons\\cropped_images"

# Connect to MySQL database
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()
query = "select max(id) from artists"
cursor.execute(query)
last_id=cursor.fetchone()[0]
if not last_id:
    last_id=0
last_id+=1
# Iterate through each file in the classical playlist folder
for filename in os.listdir(icons_folder):
            title=filename.split(".")[0]
            icon_path = os.path.join(icons_folder, filename)
            if title and not title_exists(cursor, title):
                insert_artist(cursor, last_id, title, icon_path)
                last_id+=1
            else:
                print('Skipping')

# Commit the transaction and close the connection
connection.commit()
cursor.close()
connection.close()

print("Data inserted successfully!")


Data inserted successfully!


In [27]:
import os
from PIL import Image, ImageDraw

def create_circular_crop(image_path, output_path, size):
    # Open the image
    img = Image.open(image_path).convert("RGBA")
    
    # Create a mask for the circular crop
    mask = Image.new("L", img.size, 0)
    draw = ImageDraw.Draw(mask)
    
    # Calculate the position for the circular crop
    x, y = img.size
    radius = min(x, y) // 2
    center = (x // 2, y // 2)
    
    # Draw the circle
    draw.ellipse((center[0] - radius, center[1] - radius, center[0] + radius, center[1] + radius), fill=255)
    
    # Apply the mask to the image
    circular_cropped_img = Image.new("RGBA", img.size)
    circular_cropped_img.paste(img, (0, 0), mask)

    # Resize to the desired size
    circular_cropped_img = circular_cropped_img.resize(size, Image.ANTIALIAS)

    # Save the cropped image as PNG to preserve transparency
    circular_cropped_img.save(output_path, format='PNG')

def crop_images_in_folder(input_folder, output_folder, size=(100, 100)):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Loop through each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            image_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.png")  # Save as PNG
            create_circular_crop(image_path, output_path, size)
            print(f"Cropped and saved: {output_path}")

# Example usage
input_folder = 'C:\\Users\\deeks\\Desktop\\Spotify_clone\\imgs\\artist_icons'  # Replace with your images folder path
output_folder = 'C:\\Users\\deeks\\Desktop\\Spotify_clone\\imgs\\artist_icons\\cropped_images'  # Replace with your desired output folder path

crop_images_in_folder(input_folder, output_folder, size=(150, 150))


C:\Users\deeks\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:25: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.


Cropped and saved: C:\Users\deeks\Desktop\Spotify_clone\imgs\artist_icons\cropped_images\Arijit Singh.png
Cropped and saved: C:\Users\deeks\Desktop\Spotify_clone\imgs\artist_icons\cropped_images\Jaspinder Narula.png
Cropped and saved: C:\Users\deeks\Desktop\Spotify_clone\imgs\artist_icons\cropped_images\Kavita Krishnamurthy.png
Cropped and saved: C:\Users\deeks\Desktop\Spotify_clone\imgs\artist_icons\cropped_images\KK.png
Cropped and saved: C:\Users\deeks\Desktop\Spotify_clone\imgs\artist_icons\cropped_images\Mujtaba Aziz Naza.png
Cropped and saved: C:\Users\deeks\Desktop\Spotify_clone\imgs\artist_icons\cropped_images\Osman Mir.png
Cropped and saved: C:\Users\deeks\Desktop\Spotify_clone\imgs\artist_icons\cropped_images\Pritam.png
Cropped and saved: C:\Users\deeks\Desktop\Spotify_clone\imgs\artist_icons\cropped_images\Shabab Sabri.png
Cropped and saved: C:\Users\deeks\Desktop\Spotify_clone\imgs\artist_icons\cropped_images\Shreya Goshal.png
Cropped and saved: C:\Users\deeks\Desktop\Spoti